In [2]:
# !pip install python3-nmap
# !pip install python-nmap
# !pip install graphviz

# !python.exe -m pip install python-nmap
# !python.exe -m pip install --upgrade pip
# conda install -c "hargup/label/pypi" python-nmap

import os; print(os.environ["CONDA_PREFIX"])
%conda info



  Using cached python_nmap-0.7.1-py2.py3-none-any.whl

     active environment : base
    active env location : C:\Users\malik_aa\anaconda3
       user config file : C:\Users\malik_aa\.condarc
 populated config files : C:\Users\malik_aa\.condarc
          conda version : 23.7.4
    conda-build version : 3.26.1
         python version : 3.11.5.final.0
       virtual packages : __archspec=1=x86_64
                          __cuda=12.2=0
                          __win=0=0
       base environment : C:\Users\malik_aa\anaconda3  (writable)
      conda av data dir : C:\Users\malik_aa\anaconda3\etc\conda
  conda av metadata url : None
           channel URLs : https://repo.anaconda.com/pkgs/main/win-64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.com/pkgs/r/win-64
                          https://repo.anaconda.com/pkgs/r/noarch
                          https://repo.anaconda.com/pkgs/msys2/win-64
                       

In [ ]:
# Scan the network remotely
# There is agent program that will run in individual machines and capture more detailed information of the node

import nmap, re, os
os.environ["PATH"] += os.pathsep + 'C:/Graphviz2.38/bin' # 'C:/Graphviz2.38/bin'
from graphviz import Graph
file_path, graph_file = 'analysis', 'nodes.gv'

def check_ip(ip_addr):
    regex = '''^(25[0-5]|2[0-4][0-9]|[0-1]?[0-9][0-9]?)\.( 
            25[0-5]|2[0-4][0-9]|[0-1]?[0-9][0-9]?)\.( 
            25[0-5]|2[0-4][0-9]|[0-1]?[0-9][0-9]?)\.( 
            25[0-5]|2[0-4][0-9]|[0-1]?[0-9][0-9]?)$'''
    if(re.search(regex, ip_addr)):  
        return True
          
    else:  
        return False

def networkScanner(router_ip):
    host_list=[]
    g = Graph('G', filename=file_path+'/'+graph_file, engine='sfdp')
    if check_ip(router_ip):
        print('Valid IP with length', len(router_ip))
        network = router_ip + '/24'
        print("Scanning network please wait...")
        nmap.scan(hosts=network, arguments='-sn')
        host_list = [(x, nmap[x]['status']['state']) for x in nmap.all_hosts()]
        for host, status in host_list:
            print("Found host:", host)
            if (host!=router_ip): # if host is the router, do not add an edge in the graph
                g.edge(host, router_ip)
        print('Total alive hosts:', len(host_list))
        g.view() # load node graph
    else:
        print('Invalid IP address')
    
    return host_list

nmap = nmap.PortScanner() 
print("Nmap Version: "+ str(nmap.nmap_version()[0]) +'.'+ str(nmap.nmap_version()[1]))

router_ip = input('Please provide router IP / Network ID:')
hosts = networkScanner(router_ip)

# Running os and protocols
print('Scanning hosts, please wait...')
for i in range(len(hosts)):
    ip_addr = hosts[i][0]
    print('\nCollecting information for the IP', ip_addr)
    # machine = nmap.scan(ip_addr, arguments='-p 1-65535') # Takes forever 
    machine = nmap.scan(ip_addr, arguments='-sS -O')

    
    try:
        machine_os_vendor = machine['scan'][ip_addr]['osmatch'][0]['osclass'][0]['vendor']
    except KeyError:
        machine_os_vendor = 'N/A'
        # print('KeyError machine_os_vendor', ip_addr)
    except IndexError:
        machine_os_vendor = 'N/A' 
        # print('IndexError machine_os_vendor', ip_addr)        
    try:
        machine_os = machine['scan'][ip_addr]['osmatch'][0]['osclass'][0]['osfamily']
    except KeyError:
        machine_os = 'N/A'
        # print('KeyError machine_os', ip_addr)        
    except IndexError:
        machine_os = 'N/A' 
        # print('IndexError machine_os', ip_addr)        
    try:
        machine_os_ver = machine['scan'][ip_addr]['osmatch'][0]['osclass'][0]['osgen']
    except KeyError:
        machine_os_ver = 'N/A'
        # print('KeyError machine_os_ver', ip_addr)        
    except IndexError:
        machine_os_ver = 'N/A'
        # print('IndexError machine_os_ver', ip_addr)        
    try:
        machine_os_type = machine['scan'][ip_addr]['osmatch'][0]['osclass'][0]['type']
    except KeyError:
        machine_os_type = 'N/A'
        # print('KeyError machine_os_type', ip_addr)        
    except IndexError:
        machine_os_type = 'N/A'
        # print('IndexError machine_os_type', ip_addr)        
    try:
        os_det_accuracy = machine['scan'][ip_addr]['osmatch'][0]['osclass'][0]['accuracy']
    except KeyError:
        os_det_accuracy = 'N/A'
        # print('KeyError os_det_accuracy', ip_addr)        
    except IndexError:
        os_det_accuracy = 'N/A'
        # print('IndexError os_det_accuracy', ip_addr)        

    try:
        print('Host: %s, State: %s' % (ip_addr, nmap[ip_addr].state())) # nmap[ip_addr].hostname()
        print('OS Vendor: %s, OS: %s, OS Ver: %s, OS Type: %s, Accuracy percent: %s.' % (machine_os_vendor, machine_os, machine_os_ver, machine_os_type, os_det_accuracy))
        
        if (len(nmap[ip_addr].all_protocols())>0): # if any protocol found, proceed
            for proto in nmap[ip_addr].all_protocols():
                print('Running protocol(s) : %s' % proto)
                lport = nmap[ip_addr][proto].keys()
                # lport.sort()
                for port in lport:
                    print ('port : %s\tstate : %s' % (port, nmap[ip_addr][proto][port]['state']))
    except KeyError:
        print('Node', ip_addr, 'is probably not reachable while trying to fetch details')

Nmap Version: 7.94
Please provide router IP / Network ID:192.168.0.1
Valid IP with length 11
Scanning network please wait...
Found host: 192.168.0.0
Found host: 192.168.0.1
Found host: 192.168.0.10
Found host: 192.168.0.100
Found host: 192.168.0.101
Found host: 192.168.0.102
Found host: 192.168.0.103
Found host: 192.168.0.104
Found host: 192.168.0.105
Found host: 192.168.0.106
Found host: 192.168.0.107
Found host: 192.168.0.108
Found host: 192.168.0.109
Found host: 192.168.0.11
Found host: 192.168.0.110
Found host: 192.168.0.111
Found host: 192.168.0.112
Found host: 192.168.0.113
Found host: 192.168.0.114
Found host: 192.168.0.115
Found host: 192.168.0.116
Found host: 192.168.0.117
Found host: 192.168.0.118
Found host: 192.168.0.119
Found host: 192.168.0.12
Found host: 192.168.0.120
Found host: 192.168.0.121
Found host: 192.168.0.122
Found host: 192.168.0.123
Found host: 192.168.0.124
Found host: 192.168.0.125
Found host: 192.168.0.126
Found host: 192.168.0.127
Found host: 192.168.0.12

In [ ]:
Port Data Collection: 
    Return nmap ports and services only:
        nmap 127.0.0.1 -p 1-65535 | awk 'NR >= 5 {print $1, $3}' # NR>=5 is ignore first 4 lines from results
        # ignore last 2 lines add at the end | sed '$d' | sed '$d'
        nmap 127.0.0.1 -p 1-65535 | awk 'NR >= 5 {print $1, $3}' | sed '$d' | sed '$d'
        nmap 127.0.0.1 -p 1-65535 | awk 'NR >= 5 {print $1, $3}' | sed '$d' | sed '$d' > open_ports.txt
        nmap 127.0.0.1 -p 1-65535 | awk 'NR >= 6 {print $3, $1}' | sed '$d' | sed '$d' > open_ports.txt
        nmap 127.0.0.1 -p 1-65535 | awk 'NR >= 7 {print $3":"$1}' | sed 's/\/tcp//g' | sed '$d' | sed '$d' > open_ports.txt
        https://stackoverflow.com/questions/31165747/awk-print-command-for-specific-rows

        Windows: https://ss64.com/nt/for_cmd.html
        for /f "tokens=1,3 skip=4" %i in ('nmap 127.0.0.1 -p 1-65535 ^| findstr /v "PORT"') do @echo %i %j
        for /f "tokens=1,3 skip=4" %i in ('nmap 127.0.0.1 -p 1-65535 ^| findstr /v "PORT"') do @echo %i %j >> open_ports.txt
        for /f "tokens=3,1 skip=4" %i in ('nmap 127.0.0.1 -p 1-65535 ^| findstr /v "PORT"') do @echo %j:%i >> open_ports.txt
NMAP Stealth SCAN https://linuxhint.com/stealth_scans_nmap/
    nmap 127.0.0.1 -p 1-65535
    sudo nmap -sT -p- 10.10.8.8 # SCAN ALL PORTS

Scan all IPs together:    
https://subscription.packtpub.com/book/networking_and_servers/9781783289592/4/ch04lvl1sec33/finding-open-ports    
https://www.shellhacks.com/20-nmap-examples/

netstat -a -n -o
netstat -a -n -p tcp -o

https://geekflare.com/netstat-command-usage-on-windows/
https://stackoverflow.com/questions/48198/how-can-you-find-out-which-process-is-listening-on-a-port-on-windows

   ESTABLISHED
          The socket has an established connection.
   SYN_SENT
          The socket is actively attempting to establish a connection.
   SYN_RECV
          A connection request has been received from the network.
   FIN_WAIT1
          The socket is closed, and the connection is shutting down.
   FIN_WAIT2
          Connection is closed, and the socket is waiting for  a  shutdown
          from the remote end.
   TIME_WAIT
          The socket is waiting after close to handle packets still in the
          network.
   CLOSE  The socket is not being used.
   CLOSE_WAIT
          The remote end has shut down, waiting for the socket to close.
   LAST_ACK
          The remote end has shut down, and the socket is closed.  Waiting
          for acknowledgement.
   LISTEN The  socket is listening for incoming connections.  Such sockets
          are  not  included  in  the  output  unless  you   specify   the
          --listening (-l) or --all (-a) option.
   CLOSING
          Both  sockets are shut down but we still don't have all our data
          sent.
   UNKNOWN
          The state of the socket is unknown.

Evolution CellPhone Repair: gabrielramirez1105@gmail.com
PING CODE: https://www.c-sharpcorner.com/article/python-program-to-find-all-ip-assigned-in-a-network/

In [4]:
import re
def check_ip(ip_addr):
    regex = '''^(25[0-5]|2[0-4][0-9]|[0-1]?[0-9][0-9]?)\.( 
            25[0-5]|2[0-4][0-9]|[0-1]?[0-9][0-9]?)\.( 
            25[0-5]|2[0-4][0-9]|[0-1]?[0-9][0-9]?)\.( 
            25[0-5]|2[0-4][0-9]|[0-1]?[0-9][0-9]?)$'''
    if(re.search(regex, ip_addr)):  
        return True
          
    else:  
        return False
    
print(check_ip('192.168.0-'))

False
